In [1]:
from IPython.display import display

In [2]:
import os
import sys
import shutil

import json
import yaml

from pprint import pprint

In [3]:
import pandas as pd

In [4]:
import joblib

os.environ["POLARS_MAX_THREADS"] = str(joblib.cpu_count())

In [5]:
import numpy as np
import polars as pl
import polars.datatypes as t

import rep.polars_functions as plf

In [6]:
pl.enable_string_cache()

In [7]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
%config InlineBackend.figure_format='retina'
from IPython.display import display

import seaborn as sns
import plotnine as pn
import mizani

In [8]:
from rep.notebook_init import setup_plot_style
setup_plot_style()

In [9]:
import os
import pandas as pd

import json
import yaml

import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f

import rep.spark_functions as sf
from rep.notebook_init import init_spark

spark = init_spark(
    additional_packages=["com.databricks:spark-xml_2.12:0.17.0"]
)

24/10/23 14:13:01 WARN Utils: Your hostname, ouga20.cmm.in.tum.de resolves to a loopback address: 127.0.0.1; using 192.168.16.30 instead (on interface eno8303)
24/10/23 14:13:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/data/nasif12/modules_if12/SL7/i12g/anaconda/envs/florian4/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /data/nasif12/home_if12/tsi/.ivy2/cache
The jars for the packages stored in: /data/nasif12/home_if12/tsi/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
org.xerial#sqlite-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4862f0eb-6c84-4a2e-a656-a498d0280b60;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.17.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.3.0 in central
	found org.xerial#sqlite-jdbc;3.36.0.1 in central
:: resolution report :: resolve 257ms :: artifacts dl 15ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.17.0 from central in [default]
	commons

In [10]:
spark

In [11]:
outrider_input_path = "/s/project/abexp_veff/paper_submission/zenodo_user_update1/aberrant_expression_prediction_benchmark.parquet"
vep_input_path = "/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/models/fset@DNA_only_splice_Enformer_Aparent2_isna_nosv/data.parquet"
# AbExp_input_path = "/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/predict_folds/subset=all/fset=DNA_only_splice_nosv/gtex_v8_old_dna/dna_only/DNA_only_splice_nosv@train_simplecv.py#lightgbm/data.parquet"
AbExp_input_path = "/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/predict_folds/subset=all/fset=DNA_only_splice_Enformer_Aparent2_isna_nosv/gtex_v8_old_dna/dna_only/DNA_only_splice_Enformer_Aparent2_isna_nosv@train_simplecv.py#lightgbm/data.parquet"

In [12]:
output_path="/s/project/rep/processed/training_results_v16/gtex_benchmark_with_annotation_final.parquet"

In [13]:
vep_df = spark.read.parquet(vep_input_path)
outrider_df = spark.read.parquet(outrider_input_path)
AbExp_df = (
    spark.read.parquet(AbExp_input_path)
    .select(["individual", "gene", "subtissue", "y_pred_proba", "fold"])
    .withColumnRenamed("y_pred_proba", "AbExp")
)

In [14]:
columns_to_keep = [
    "individual",
    "gene",
    "tissue",
    "AbExp",
    "cadd_raw",
    "fold",
    "outlier_state",
    "theta",
    "zscore"
]

In [15]:
(
    AbExp_df
    .filter(f.col("individual") == "GTEX-148VJ")
    .filter(f.col("gene") == "ENSG00000124333")
    .filter(f.col("subtissue") == "Prostate")
    .toPandas()
)

,individual,gene,subtissue,AbExp,fold
0,GTEX-148VJ,ENSG00000124333,Prostate,-0.002782,1


In [16]:
AbExp_df.groupby(["individual","gene","subtissue"]).count().sort("count").toPandas()

,individual,gene,subtissue,count
0,GTEX-113JC,ENSG00000007341,Cells - Cultured fibroblasts,1
1,GTEX-113JC,ENSG00000173960,Cells - Cultured fibroblasts,1
2,GTEX-113JC,ENSG00000121210,Esophagus - Gastroesophageal Junction,1
3,GTEX-113JC,ENSG00000108389,Skin - Sun Exposed (Lower leg),1
4,GTEX-113JC,ENSG00000075089,Uterus,1
...,...,...,...,...
99459912,GTEX-ZE7O,ENSG00000174718,Stomach,1
99459913,GTEX-ZTSS,ENSG00000053770,Thyroid,1
99459914,GTEX-ZYFG,ENSG00000116106,Artery - Aorta,1
99459915,GTEX-ZYFG,ENSG00000134575,Artery - Coronary,1


In [17]:
features_df = AbExp_df.join(
    vep_df.drop(*[
        "fold",
        "theta",
        "zscore",
    ]),
    on=["individual","gene","subtissue"],
    how="left"
)
# using != True also handles NULL values
features_df = (
    features_df
    .withColumnRenamed("subtissue", "tissue")
    .select([
        "individual",
        "gene",
        "tissue",
        "AbExp",
        *[f.col(f"`{c}`").alias(c[8:].replace("features.", "")) for c in vep_df.columns if c.startswith("feature")],
        # "is_obvious_outlier",
    ])
)

joined = (
    outrider_df
    .join(features_df, on=["individual", "gene", "tissue"], how="left")
    .filter(f.col("is_in_benchmark"))
    .sort([
        "individual",
        "gene",
        "tissue_type",
        "tissue",
    ])
    .persist()
)

24/10/23 14:14:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [18]:
joined.printSchema()

root
 |-- individual: string (nullable = true)
 |-- gene: string (nullable = true)
 |-- tissue: string (nullable = true)
 |-- tissue_type: string (nullable = true)
 |-- mu: float (nullable = true)
 |-- theta: float (nullable = true)
 |-- counts: integer (nullable = true)
 |-- normalized_counts: double (nullable = true)
 |-- l2fc: float (nullable = true)
 |-- zscore: float (nullable = true)
 |-- nominal_pvalue: double (nullable = true)
 |-- FDR: double (nullable = true)
 |-- is_in_benchmark: boolean (nullable = true)
 |-- is_underexpressed_outlier: boolean (nullable = true)
 |-- AbExp: double (nullable = true)
 |-- aparent2@Aparent2: double (nullable = true)
 |-- aparent2@is_near_pas: boolean (nullable = true)
 |-- enformer@Enformer: float (nullable = true)
 |-- splice_ensemble@AbSplice: double (nullable = true)
 |-- splice_ensemble@MMSplice_SpliceMap_Psi_ref: double (nullable = true)
 |-- splice_ensemble@SpliceAI: double (nullable = true)
 |-- vep@cadd_raw.max: float (nullable = true)


In [19]:
joined.write.parquet(output_path, mode="overwrite")

In [20]:
spark.read.parquet(output_path).groupby([
    "is_underexpressed_outlier",
]).count().toPandas()

,is_underexpressed_outlier,count
0,True,11200
1,False,99448717
